<a href="https://colab.research.google.com/github/hassanSattariNia/prediction_time_series/blob/main/FL-Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch peft scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import torch

In [3]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
)

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
model = get_peft_model(model, lora_config)

# بارگذاری دیتاست
dataset = load_dataset("glue", "mrpc")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
train_dataset = tokenized_dataset['train']
train_dataset_split = train_dataset.train_test_split(test_size=0.5, seed=42)

train_dataset1 = train_dataset_split['train']
train_dataset2 = train_dataset_split['test']


In [10]:
def federated_learning(num_rounds=3):
    # ایجاد مدل‌ها و کلاینت‌ها
    model1 = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
    model2 = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
    model1 = get_peft_model(model1, lora_config)
    model2 = get_peft_model(model2, lora_config)

    client1 = Client(model1, train_dataset1)
    client2 = Client(model2, train_dataset2)

    server = Server(model1)  # سرور از مدل اول استفاده می‌کند

    for round in range(num_rounds):
        print(f"Round {round + 1}")

        # آموزش کلاینت‌ها
        client1.train()
        client2.train()

        # جمع‌آوری پارامترها از کلاینت‌ها
        client_params = [client1.get_model_parameters(), client2.get_model_parameters()]

        # تجمیع پارامترها در سرور
        aggregated_params = server.aggregate_parameters(client_params)

        # به‌روزرسانی پارامترها در سرور و کلاینت‌ها
        server.update_model_parameters(aggregated_params)
        client1.update_model_parameters(aggregated_params)
        client2.update_model_parameters(aggregated_params)

    # ارزیابی نهایی
    final_model = server.model
    trainer = Trainer(
        model=final_model,
        eval_dataset=tokenized_dataset['validation'],
        tokenizer=tokenizer,
    )
    results = trainer.evaluate()
    print("Final Evaluation Results:", results)

In [8]:
class Client:
    def __init__(self, model, dataset):
        self.model = model
        self.dataset = dataset
        self.trainer = None

    def train(self):
        training_args = TrainingArguments(
            output_dir="./results",
            num_train_epochs=1,
            per_device_train_batch_size=16,
            logging_dir="./logs",
            logging_steps=500,
            save_total_limit=1
        )
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.dataset,
            tokenizer=tokenizer,
        )
        self.trainer.train()

    def get_model_parameters(self):
        return self.model.state_dict()

    def update_model_parameters(self, new_params):
        self.model.load_state_dict(new_params)

In [9]:
class Server:
    def __init__(self, model):
        self.model = model

    def aggregate_parameters(self, client_parameters):
        aggregated_params = {}
        for key in client_parameters[0].keys():
            aggregated_params[key] = sum(client_param[key] for client_param in client_parameters) / len(client_parameters)
        return aggregated_params

    def update_model_parameters(self, new_params):
        self.model.load_state_dict(new_params)

    def get_model_parameters(self):
        return self.model.state_dict()

In [11]:
federated_learning()

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Round 1


Step,Training Loss


Step,Training Loss


Round 2


...

Step,Training Loss


Round 3


Step,Training Loss


Step,Training Loss


Final Evaluation Results: {'eval_model_preparation_time': 0.0013, 'eval_runtime': 14.7144, 'eval_samples_per_second': 27.728, 'eval_steps_per_second': 3.466}
